In [1]:
# Import the dependencies.
import pandas as pd
import numpy as np
import time

In [2]:
import os
import sys

# Method to add the previous dir to the path found at:
# https://stackoverflow.com/questions/34478398/import-local-function-from-a-module-housed-in-another-directory-with-relative-im/35273613#35273613

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

In [3]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=ed5c4a5d13114591e5ee76b3bbaf9589


In [4]:
# Create a set of random latitude and longitude combinations. zip packs the data in pairs
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [5]:
# Add the latitudes and longitudes to a list. This command is unpacking the zip array into a tuple list, 
# you can only do this once as it then removes it from memory
coordinates = list(lat_lngs)

In [6]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

758

In [7]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city. city.replace(" ","+") takes out the spaces and concatonates the name together
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_description = city_weather["weather"][0]["description"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | trat
Processing Record 2 of Set 1 | butaritari
Processing Record 3 of Set 1 | ushuaia
Processing Record 4 of Set 1 | avarua
Processing Record 5 of Set 1 | nanortalik
Processing Record 6 of Set 1 | grindavik
Processing Record 7 of Set 1 | hilo
Processing Record 8 of Set 1 | xining
Processing Record 9 of Set 1 | ankazoabo
Processing Record 10 of Set 1 | dikson
Processing Record 11 of Set 1 | mariel
Processing Record 12 of Set 1 | buraydah
Processing Record 13 of Set 1 | kaitangata
Processing Record 14 of Set 1 | thompson
Processing Record 15 of Set 1 | matamoros
Processing Record 16 of Set 1 | cockburn town
Processing Record 17 of Set 1 | atuona
Processing Record 18 of Set 1 | port elizabeth
Processing Record 19 of Set 1 | airai
Processing Record 20 of Set 1 | krasnyy chikoy
Processing Record 21 of Set 1 | meulaboh
Processing Record 22 of Set 1 | carnarvon
Processing Record 23 of Set 1 | la asuncio

Processing Record 40 of Set 4 | attawapiskat
City not found. Skipping...
Processing Record 41 of Set 4 | chokurdakh
Processing Record 42 of Set 4 | pangnirtung
Processing Record 43 of Set 4 | rawson
Processing Record 44 of Set 4 | torbay
Processing Record 45 of Set 4 | yumen
Processing Record 46 of Set 4 | srednekolymsk
Processing Record 47 of Set 4 | palabuhanratu
City not found. Skipping...
Processing Record 48 of Set 4 | dinsor
City not found. Skipping...
Processing Record 49 of Set 4 | rabo de peixe
Processing Record 50 of Set 4 | pailon
Processing Record 1 of Set 5 | buchanan
Processing Record 2 of Set 5 | sorland
Processing Record 3 of Set 5 | la plata
Processing Record 4 of Set 5 | carutapera
Processing Record 5 of Set 5 | kiama
Processing Record 6 of Set 5 | kenai
Processing Record 7 of Set 5 | melbourne
Processing Record 8 of Set 5 | fairmont
Processing Record 9 of Set 5 | novosergiyevka
Processing Record 10 of Set 5 | cherskiy
Processing Record 11 of Set 5 | sao joao da barra

Processing Record 31 of Set 8 | vanavara
Processing Record 32 of Set 8 | tazovskiy
Processing Record 33 of Set 8 | yingcheng
Processing Record 34 of Set 8 | nizhniy odes
Processing Record 35 of Set 8 | freeport
Processing Record 36 of Set 8 | oktyabrskiy
Processing Record 37 of Set 8 | odweyne
Processing Record 38 of Set 8 | nsunga
Processing Record 39 of Set 8 | tabas
Processing Record 40 of Set 8 | lowestoft
Processing Record 41 of Set 8 | kendari
Processing Record 42 of Set 8 | hamilton
Processing Record 43 of Set 8 | yakeshi
Processing Record 44 of Set 8 | haimen
Processing Record 45 of Set 8 | kure
Processing Record 46 of Set 8 | kindu
Processing Record 47 of Set 8 | annau
Processing Record 48 of Set 8 | komsomolskiy
Processing Record 49 of Set 8 | samarai
Processing Record 50 of Set 8 | dehloran
Processing Record 1 of Set 9 | amderma
City not found. Skipping...
Processing Record 2 of Set 9 | wanning
Processing Record 3 of Set 9 | bandundu
Processing Record 4 of Set 9 | piacabucu


Processing Record 23 of Set 12 | acapulco
Processing Record 24 of Set 12 | tinskoy
Processing Record 25 of Set 12 | rundu
Processing Record 26 of Set 12 | walvis bay
Processing Record 27 of Set 12 | port hawkesbury
Processing Record 28 of Set 12 | massakory
Processing Record 29 of Set 12 | ngukurr
City not found. Skipping...
Processing Record 30 of Set 12 | azimur
City not found. Skipping...
Processing Record 31 of Set 12 | arbuzynka
Processing Record 32 of Set 12 | tres arroyos
Processing Record 33 of Set 12 | sumbe
Processing Record 34 of Set 12 | barkhan
Processing Record 35 of Set 12 | maceio
Processing Record 36 of Set 12 | ruangwa
Processing Record 37 of Set 12 | asau
Processing Record 38 of Set 12 | horsham
Processing Record 39 of Set 12 | hurricane
Processing Record 40 of Set 12 | bima
Processing Record 41 of Set 12 | lardos
Processing Record 42 of Set 12 | saint-pierre
Processing Record 43 of Set 12 | seddon
Processing Record 44 of Set 12 | sioux lookout
Processing Record 45 o

-----------------------------
Data Retrieval Complete      
-----------------------------


In [8]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,Trat,12.5000,102.5000,88.00,54,91,9.89,TH,overcast clouds
1,Butaritari,3.0707,172.7902,81.57,80,100,17.13,KI,light rain
2,Ushuaia,-54.8000,-68.3000,37.06,81,40,25.32,AR,scattered clouds
3,Avarua,-21.2078,-159.7750,84.25,79,100,9.22,CK,overcast clouds
4,Nanortalik,60.1432,-45.2371,30.61,83,50,3.20,GL,scattered clouds
5,Grindavik,63.8424,-22.4338,28.87,69,40,25.32,IS,scattered clouds
6,Hilo,19.7297,-155.0900,76.73,79,100,6.91,US,overcast clouds
7,Xining,36.6167,101.7667,49.30,29,0,2.10,CN,clear sky
8,Ankazoabo,-22.2833,44.5167,67.37,79,0,4.54,MG,clear sky
9,Dikson,73.5069,80.5464,-0.18,80,14,19.24,RU,few clouds


In [9]:
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp","Humidity","Cloudiness","Wind Speed","Current Description"]
city_data_df=city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Trat,TH,12.5000,102.5000,88.00,54,91,9.89,overcast clouds
1,Butaritari,KI,3.0707,172.7902,81.57,80,100,17.13,light rain
2,Ushuaia,AR,-54.8000,-68.3000,37.06,81,40,25.32,scattered clouds
3,Avarua,CK,-21.2078,-159.7750,84.25,79,100,9.22,overcast clouds
4,Nanortalik,GL,60.1432,-45.2371,30.61,83,50,3.20,scattered clouds
5,Grindavik,IS,63.8424,-22.4338,28.87,69,40,25.32,scattered clouds
6,Hilo,US,19.7297,-155.0900,76.73,79,100,6.91,overcast clouds
7,Xining,CN,36.6167,101.7667,49.30,29,0,2.10,clear sky
8,Ankazoabo,MG,-22.2833,44.5167,67.37,79,0,4.54,clear sky
9,Dikson,RU,73.5069,80.5464,-0.18,80,14,19.24,few clouds


In [10]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Databse.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")